## Librerías

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import keras
import PIL
from PIL import Image, ImageShow
from sklearn import preprocessing

import time
from IPython.display import clear_output
import os

In [2]:
from PIL import GifImagePlugin
GifImagePlugin.LOADING_STRATEGY = GifImagePlugin.LoadingStrategy.RGB_ALWAYS

PIL.ImageShow.register(PIL.ImageShow.IPythonViewer,-1)

## Lectura de datos

In [3]:
labels = np.load("dict.npy", allow_pickle = True).tolist()
labels_inv = dict((v, k) for k, v in labels.items())

In [4]:
img_path = "./Train/"
img_names = os.listdir(img_path)
train_labels = pd.read_csv("train.csv").sort_values(by=['Id'])

In [5]:
train_labels["Familia"] = train_labels["Expected"].apply(lambda x : labels[x[0]])
train_labels["Animal"] = train_labels["Expected"].apply(lambda x : labels[x[-2:].strip()])

In [6]:
df = pd.get_dummies(train_labels, columns = ["Familia", "Animal"])
df = df.reset_index()
df.columns = list(map(lambda x : x.replace("Familia_", "").replace("Animal_", ""), df.columns))
df = df.drop(['index', 'Id', 'Unnamed: 0', 'Expected', 'Anfibio', 'Artropodo', 'Ave', 'Mamifero', 'Reptil'], axis = 1)

In [ ]:
img_size=(224,224)
images = list()
for img in img_names:
    image = Image.open(img_path + img).convert("RGB")
    images.append(np.asarray(image.resize(img_size)))
images = np.array(images)
images.shape

## Data Augmentation

In [7]:
file = open("data224.obj", "rb")
images = pickle.load(file)
file.close()

In [8]:
animales = dict()
for col in df.columns:
    animales[col] = images[df[df[col]==1].index]

In [9]:
augment = keras.Sequential([
    tf.keras.layers.Input((224,224,3)),
    tf.keras.layers.RandomFlip(mode = "horizontal"),
    tf.keras.layers.RandomRotation(.1),
    tf.keras.layers.RandomZoom((-0.2, 0.2)),
    tf.keras.layers.RandomTranslation((-0.05, 0.05),(-0.05, 0.05)),
    tf.keras.layers.RandomContrast((.1,.3)),
    tf.keras.layers.GaussianNoise(5)
])

In [174]:
for col in df.columns[:]:
    try:
        os.remove(f"./Augmented/{col}/XY_{col}.obj")
        os.remove(f"./Augmented/{col}/aug_{col}.obj")
    except:
        continue

In [175]:
for col in df.columns:
    data = animales[col]
    i = 0
    while len(data) < 1200:
        aug = augment(animales[col], training = True)
        data = np.append(data, aug, axis = 0)
        print(f"{col}, {i}")
        print(data.shape)
        clear_output(wait = True)
        i += 1
    file = open(f"./Augmented/{col}/aug_{col}.obj", "wb")
    pickle.dump(data, file)
    file.close()
    del data

Viuda negra, 12
(1246, 224, 224, 3)


In [177]:
for col in df.columns:
    print(col)
    file = open(f"./Augmented/{col}/aug_{col}.obj", "rb")
    temp_1 = pickle.load(file)
    file.close()
    temp_2 = np.empty((1, 224, 224, 3))
    for _ in df.columns:
        if not _ == col:
            temp_2 = np.concatenate((temp_2, animales[_]), axis = 0)
    print(temp_1.shape, temp_2.shape)
    Y = np.concatenate((np.ones(temp_1.shape[0]), np.zeros(temp_2.shape[0])), axis=0)
    X = np.concatenate((temp_1, temp_2), axis=0)
    file = open(f"./Augmented/{col}/XY_{col}.obj", "wb")
    pickle.dump((X,Y), file)
    file.close()
    del temp_1, temp_2
    clear_output(wait = True)

Viuda negra
(1246, 224, 224, 3) (3433, 224, 224, 3)


## Entrenamiento

In [4]:
def create_model(base_model):
    inputs = keras.layers.Input((224,224,3))
    x = keras.layers.RandomFlip(mode = "horizontal")(inputs)
    x = keras.layers.RandomRotation(.05)(x)
    x = tf.keras.layers.RandomZoom((-0.2, 0))(x)
    x = keras.layers.Rescaling(scale = 1 / 127.5, offset = -1)(inputs)
    x = base_model(x, training = False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(86)(x)
    outputs = keras.layers.Dense(1,activation = "sigmoid")(x)
    model = keras.Model(inputs, outputs)
    return model

In [5]:
base_model = keras.applications.InceptionV3(input_shape = (224,224,3), weights = 'imagenet', include_top = False)
base_model.trainable = False
model = create_model(base_model)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 86)                176214    
                                                                 
 dense_1 (Dense)             (None, 1)                 87        
                                                             

In [29]:
epoch = 50
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            restore_best_weights=True)

In [34]:
i = 0
for col in df.columns[8:]:
    print(f"{i}/32")
    file = open(f"./Augmented/{col}/XY_{col}.obj", "rb")
    X,Y = pickle.load(file)
    file.close()
    base_model = keras.applications.InceptionV3(input_shape = (224,224,3), weights = 'imagenet', include_top = False)
    base_model.trainable = False
    model = create_model(base_model)
    model.compile(optimizer = keras.optimizers.Adam(0.01), 
                  loss = keras.losses.BinaryCrossentropy(), 
                  metrics = ["binary_accuracy"]
                 )
    model.fit(X, Y, epochs = epoch, callbacks = callback, validation_split = 0.2)
    model.save(f"./Augmented/{col}/model_{col}")
    model.save_weights(f"./Augmented/{col}/model_{col}_weights")
    #base_model.trainable = True
    #model.compile(optimizer = keras.optimizers.Adam(1e-7), 
    #              loss = keras.losses.BinaryCrossentropy(), 
    #              metrics = ["binary_accuracy"]
    #             )
    #model.fit(X, Y, epochs = epoch, callbacks = callback, validation_split = 0.2)
    #model.save(f"./Augmented/{col}/model_{col}_wtuning")
    #model.save_weights(f"./Augmented/{col}/model_{col}_wtuning_weights")
    clear_output(wait = True)
    i += 1

23/32
Epoch 1/50
117/117 [==============================] - 125s 1s/step - loss: 0.0302 - binary_accuracy: 0.9933 - val_loss: 4.0497e-05 - val_binary_accuracy: 1.0000
Epoch 2/50
117/117 [==============================] - 118s 1s/step - loss: 0.0180 - binary_accuracy: 0.9987 - val_loss: 2.0128 - val_binary_accuracy: 0.9348
Epoch 3/50
117/117 [==============================] - 119s 1s/step - loss: 0.0015 - binary_accuracy: 0.9997 - val_loss: 0.0000e+00 - val_binary_accuracy: 1.0000
Epoch 4/50
117/117 [==============================] - 119s 1s/step - loss: 4.8375e-09 - binary_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_binary_accuracy: 1.0000
Epoch 5/50
117/117 [==============================] - 119s 1s/step - loss: 4.7635e-09 - binary_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_binary_accuracy: 1.0000
Epoch 6/50
117/117 [==============================] - 119s 1s/step - loss: 4.6786e-09 - binary_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_binary_accuracy: 1.0000
Epoch 7/50
117/117 [

INFO:tensorflow:Assets written to: ./Augmented/Viuda negra/model_Viuda negra\assets


INFO:tensorflow:Assets written to: ./Augmented/Viuda negra/model_Viuda negra\assets


## Predicciones

In [35]:
test_path = "./Test/"
test_names = os.listdir(test_path)
output = pd.DataFrame(test_names, columns = ["Id"])

In [37]:
test = list()
for img in test_names:
    _ = Image.open(test_path + img).convert("RGB")
    test.append(np.asarray(_.resize((224,224))))
test = np.array(test)
test.shape

(1174, 224, 224, 3)

In [38]:
for col in df.columns:
    print(col)
    model = keras.models.load_model(f"./Augmented/{col}/model_{col}")
    pred = model.predict(test)
    output[col] = pred
    clear_output(wait = True)

Viuda negra
37/37 [==============================] - 54s 1s/step


In [64]:
for col in df.columns:
    print(f"{col}:{t[t==col].count()}")

Buho:47
Caballo:25
Cabra:22
Cacique:35
Camaleon:21
Cisne:44
Cocodrilo:55
Conejo:51
Cormoran:37
Elefante:41
Ganso:43
Gato:32
Iguana:27
Lagarto:55
Leon:34
Lobo:36
Lombriz:23
Mariposa:42
Oso:31
Paloma:36
Pato:25
Pavo:38
Perro:39
Rana:48
Salamandra:23
Saltamontes:39
Sapo:48
Serpiente:30
Tarantula:26
Tigre:31
Tortuga:57
Viuda negra:33


In [94]:
Anfibio = ['5', '6', '7']
Artropodo = ['8', '9', '10', '11', '12']
Ave = ['13', '14', '15', '16', '17', '18', '19', '20']
Mamifero = ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
Reptil = ['31', '32', '33', '34', '35', '36']
clases = ['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36']

labels_format = dict()
for i in clases:
    if i in Anfibio:
        labels_format[i] = f"0 {i}"
    elif i in Artropodo:
        labels_format[i] = f"1 {i}"
    elif i in Ave:
        labels_format[i] = f"2 {i}"
    elif i in Mamifero:
        labels_format[i] = f"3 {i}"
    elif i in Reptil:
        labels_format[i] = f"4 {i}"

In [105]:
submission = pd.DataFrame(output["Id"], columns=["Id"])
submission["Expected"] = output.drop(columns = ["Id"]).idxmax(axis = 1)
submission["Expected"] = list(map(lambda x: labels_format[labels_inv[x]], submission["Expected"]))
submission.to_csv("Test_submit.csv", index=False)

In [106]:
submission

,Id,Expected
0,image_100.jpg,0 5
1,image_1001.jpg,1 12
2,image_1010.jpg,1 12
3,image_1018.jpg,1 12
4,image_1020.jpg,1 12
...,...,...
1169,image_978.jpg,1 12
1170,image_99.jpg,0 5
1171,image_994.jpg,1 12
1172,image_996.jpg,1 12
